# 🦩 Using Ibis-framework in Kaggle 🦩

![ibis](https://camo.qiitausercontent.com/536834ed6cbfe4ed6623b4b395baedcc31ce84a0/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3639323136352f39316330333737302d623434302d363438632d633633372d3631616666343465656533362e706e67)

Do you know about Ibis ? Ibis is a library that provides an integrated interface for data processing, allowing you to use over 18 supported data processing libraries with the same syntax. As of January 2024, it supports the following frameworks.

BigQuery , ClickHouse , Dask , DataFusion , Druid ,
DuckDB , Exasol ,
Flink , Impala , MSSQL , MySQL , Oracle , pandas ( CuDF ) , Polars ,
PostresSQL , PySpark , Snowflake , SQLite , Trino

![ibis2](https://camo.qiitausercontent.com/4ce38b715f22e9c3947631bacf4a3e95361726bf/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3639323136352f34393836646236322d656161332d656136372d613462662d3761636362393039323134352e706e67)

With Ibis, you can change the backend framework anytime without having to rewrite your code.

In [ ]:
# %load_ext cudf.pandas # you can also use cudf

ibis.set_backend("pandas") # Set pandas as the backend

t = (
    ibis.read_csv("titanic.csv")
    .select("name", "sex", "age", "fare")
    .filter(t["sex"] == "female")
    .mutate(
        # Calculate z-scores for 'age' and 'fare'
        s.across(["age", "fare"], {"zscore": lambda x: ((x - x.mean()) / x.std()) * 10 + 50}))
    .order_by(ibis.desc("age")) # Sort by 'age' column in descending order
    )

t.execute() # Execute the query

In [ ]:
ibis.set_backend("polars") # Set pandas as the backend

t = (
    ibis.read_csv("titanic.csv")
    .select("name", "sex", "age", "fare")
    .filter(t["sex"] == "female")
    .mutate(
        # Calculate z-scores for 'age' and 'fare'
        s.across(["age", "fare"], {"zscore": lambda x: ((x - x.mean()) / x.std()) * 10 + 50}))
    .order_by(ibis.desc("age")) # Sort by 'age' column in descending order
    )

t.execute() # Execute the query

Just like in the code example above, you can switch the backend processing engine from pandas to polars by simply modifying only the first line.

If you are interested in Ibis, you can study it through the following content.

https://github.com/kunishou/Ibis_100_knocks

This time, I tried using Ibis on Kaggle. I referred to the following notebook for guidance. Thank you :)

https://www.kaggle.com/code/docxian/hms-harmful-brain-activity-first-glance

## Table of Contents
* [File Overview](#files)
* [Training File](#train)
* [EEG File Example](#ex_EEG)
* [Spectrogram File Example](#ex_spec)
* [Test and Submission File](#sub)

In [ ]:
# install ibis-framework
!pip install /kaggle/input/ibis-framework/*.whl -qq 2>/dev/null

In [ ]:
# packages

# standard
import numpy as np
import pandas as pd
import ibis
import time

# plot
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ibis.options.interactive = True # Ibis Eager Mode

In [ ]:
ibis.set_backend("polars") # set polars as backend engine 
# ibis.set_backend("pandas")
# ibis.set_backend("duckdb")

In [ ]:
# configs
ibis.options.repr.interactive.max_rows = 100 # we want to display 100 columns in this notebook

# aesthetics
default_color_1 = 'darkblue'
default_color_2 = 'darkgreen'
default_color_3 = 'darkred'

<a id='files'></a>
# File Overview

In [ ]:
!ls -l '../input/hms-harmful-brain-activity-classification'

In [ ]:
!ls -l '../input/hms-harmful-brain-activity-classification/example_figures'

In [ ]:
!ls -l '../input/hms-harmful-brain-activity-classification/train_spectrograms'

In [ ]:
!ls -l '../input/hms-harmful-brain-activity-classification/train_eegs'

<a id='train'></a>
# Training File

In [ ]:
t_train = ibis.read_csv("../input/hms-harmful-brain-activity-classification/train.csv")
t_train.head()

In [ ]:
# votes columns
features_vote = ['seizure_vote', 'lpd_vote', 'gpd_vote',
                 'lrda_vote', 'grda_vote', 'other_vote']

In [ ]:
# Group by eeg_id and calculate the mean of the vote columns
t_train_unique_votes = t_train.group_by('eeg_id').agg(
    [t_train[col].mean().name(col) for col in features_vote]
)

# Calculate the sum of votes for each row
t_train_unique_votes = t_train_unique_votes.mutate(
    vote_sum=sum(t_train_unique_votes[col] for col in features_vote)
)

# Normalize votes to get a 100% distribution for each row/EEG
for f in features_vote:
    t_train_unique_votes = t_train_unique_votes.mutate(
        **{f: t_train_unique_votes[f] / t_train_unique_votes.vote_sum}
    )
    
t_train_unique_votes.head(10)

<a id='ex_EEG'></a>
# EEG File Example

In [ ]:
# load an EEG file
t_eeg = ibis.read_parquet('../input/hms-harmful-brain-activity-classification/train_eegs/1000913311.parquet')

In [ ]:
# preview
t_eeg.head()

In [ ]:
# show structure
t_eeg.info()

In [ ]:
# basis stats
t_eeg.execute().describe()

In [ ]:
# list of features
features_eeg = ['Fp1', 'F3', 'C3', 'P3', 'F7', 
                'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz',
                'Fp2', 'F4', 'C4', 'P4', 'F8',
                'T4', 'T6', 'O2', 'EKG']

In [ ]:
# plot all features
for f in features_eeg:
    plt.figure(figsize=(12,3))
    plt.plot(t_eeg[f].execute(), color=default_color_1)
    plt.title(f)
    plt.grid()
    plt.show()

In [ ]:
cor_eeg = t_eeg[features_eeg].execute().corr(method='pearson')
plt.figure(figsize=(12,8))
sns.heatmap(cor_eeg, annot=True,
            fmt='.2f',
            linecolor='black', linewidths=.5,
            cmap='RdYlGn', vmin=-1, vmax=+1)
plt.title('Correlation - EEG example')
plt.show()

<a id='ex_spec'></a>
# Spectrogram File Example

In [ ]:
# load spectrogram file
t_spec = ibis.read_parquet('../input/hms-harmful-brain-activity-classification/train_spectrograms/1000086677.parquet')

In [ ]:
# preview
t_spec.head()

In [ ]:
# show structure
t_spec.execute().info(verbose=True, show_counts=True)

In [ ]:
# basis stats
t_spec.execute().describe()

### Some plots:

In [ ]:
# plotting function
def plot_spec(i_frequency):
    feature_1 = 'LL_' + i_frequency
    feature_2 = 'RL_' + i_frequency
    feature_3 = 'RP_' + i_frequency
    feature_4 = 'LP_' + i_frequency
    plt.figure(figsize=(10,3))
    plt.scatter(df_spec.time, df_spec[feature_1], label='LL')
    plt.scatter(df_spec.time, df_spec[feature_2], label='RL')
    plt.scatter(df_spec.time, df_spec[feature_3], label='RP')
    plt.scatter(df_spec.time, df_spec[feature_4], label='LP')
    plt.legend(loc='upper right')
    plt.title('Frequency=' + i_frequency)
    plt.grid()
    plt.show()

In [ ]:
df_spec = t_spec.execute()

# plot for a few frequencies
frequencies = ['0.59', '3.91', '10.16', '19.92']
for freq in frequencies:
    plot_spec(freq)

<a id='sub'></a>
# Test and Submission File

In [ ]:
t_test = ibis.read_csv('../input/hms-harmful-brain-activity-classification/test.csv')
t_test

In [ ]:
t_sub = ibis.read_csv('../input/hms-harmful-brain-activity-classification/sample_submission.csv')
t_sub

#### 

#### Simply use (grouped) means for a first baseline. See also this notebook: https://www.kaggle.com/code/seshurajup/eda-train-csv

In [ ]:
mean_values = t_train_unique_votes.agg(
    t_train_unique_votes[col].mean().name(col) for col in features_vote
).execute()
mean_values

In [ ]:
t_sub = t_sub.mutate(
    seizure_vote=float(mean_values['seizure_vote']),
    lpd_vote=float(mean_values['lpd_vote']),
    gpd_vote=float(mean_values['gpd_vote']),
    lrda_vote=float(mean_values['lrda_vote']),
    grda_vote=float(mean_values['grda_vote']),
    other_vote=float(mean_values['other_vote'])
)
t_sub

In [ ]:
t_sub.execute()

In [ ]:
# save submission file
t_sub.execute().to_csv('submission.csv', index=False)

### Work in progress